# K nearest neighbors

Implementaremos el algoritmo que vimos en la teoría, utilizando sklearn.

Sklearn tiene una implementación para KNN classifier: [documentación KNNClassifier](https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsClassifier.html)

Y otra para el regressor: [documentación KNNRegressor](
https://scikit-learn.org/stable/modules/generated/sklearn.neighbors.KNeighborsRegressor.html)




In [1]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt

Vamos a trabajar con el dataset penguins que podemos cargar desde seaborn.

La idea es que entrenemos un KNN para clasificar pinguinos (predecir la variable species)

In [2]:
df = sns.load_dataset("penguins")

In [3]:
df.head()

,species,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Adelie,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Adelie,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Adelie,Torgersen,40.3,18.0,195.0,3250.0,Female
3,Adelie,Torgersen,NaN,NaN,NaN,NaN,NaN
4,Adelie,Torgersen,36.7,19.3,193.0,3450.0,Female


¿ Hay nulos ?

In [4]:
df.isna().sum()

species               0
island                0
bill_length_mm        2
bill_depth_mm         2
flipper_length_mm     2
body_mass_g           2
sex                  11
dtype: int64

En caso de haberlos, por simplicidad los vamos a descartar.

Descartar nulos:

In [66]:
df = df.dropna()

In [67]:
df.isna().sum()

species              0
island               0
bill_length_mm       0
bill_depth_mm        0
flipper_length_mm    0
body_mass_g          0
sex                  0
dtype: int64

Dividimos en X e y

In [68]:
X = df.drop("species", axis=1)
y = df["species"].copy()

In [8]:
X.head()

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
0,Torgersen,39.1,18.7,181.0,3750.0,Male
1,Torgersen,39.5,17.4,186.0,3800.0,Female
2,Torgersen,40.3,18.0,195.0,3250.0,Female
4,Torgersen,36.7,19.3,193.0,3450.0,Female
5,Torgersen,39.3,20.6,190.0,3650.0,Male


In [9]:
y.head()

0    Adelie
1    Adelie
2    Adelie
4    Adelie
5    Adelie
Name: species, dtype: object

¿ Cuántos pinguinos tenemos de cada especie ?



In [10]:
y.value_counts()

Adelie       146
Gentoo       119
Chinstrap     68
Name: species, dtype: int64

¿ Y en porcentajes ?

In [11]:
y.value_counts(normalize=True)

Adelie       0.438438
Gentoo       0.357357
Chinstrap    0.204204
Name: species, dtype: float64

## Baseline

¿Cómo se les ocurre definir un baseline para este caso?

No hay una única manera correcta, tiene que ser un modelo simple.



In [12]:
# COMPLETAR

El modelo que desarrollemos, tiene que ser mejor que este baseline. ¿ Qué accuracy_score tiene el baseline ?

In [13]:
from sklearn.metrics import accuracy_score

# MEDIR ACCURACY_SCORE DEL BASELINE

## Train - test split

Como vimos la clase anterior, es importante guardarnos un conjunto de test para evaluar el modelo.

Vamos a hacer un train-test split utilizando sklearn.

primero, importar train_test_split de sklearn:

In [14]:
from sklearn.model_selection import train_test_split

Aplicar la función para obtener: X_train, X_test, y_train e y_test.

Vamos a tomar un 15% de los datos para el conjunto de test. Como las clases no están balanceadas, sería bueno utilizar el stratify que nos provee sklearn (vimos en el notebook de la clase pasada)

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.15, stratify=y, random_state=42)

In [51]:
X_train = X_train.copy()

In [52]:
X_train.shape

(283, 6)

In [53]:
X_test.shape

(50, 6)

In [54]:
y_train.shape

(283,)

In [55]:
y_test.shape

(50,)

## Preprocesamiento de datos

Vimos que en KNN es muy importante que los datos estén en una misma escala.

¿En que rango de valores se encuentran las variables numéricas del dataset?

In [56]:
df.describe()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g
count,333.000000,333.000000,333.000000,333.000000
mean,43.992793,17.164865,200.966967,4207.057057
std,5.468668,1.969235,14.015765,805.215802
min,32.100000,13.100000,172.000000,2700.000000
25%,39.500000,15.600000,190.000000,3550.000000
50%,44.500000,17.300000,197.000000,4050.000000
75%,48.600000,18.700000,213.000000,4775.000000
max,59.600000,21.500000,231.000000,6300.000000


Debemos llevar todo a una misma escala. Para esto utilizaremos el StandardScaler de sklearn.

Importar standard scaler:

In [57]:
from sklearn.preprocessing import StandardScaler

Crear una instancia de StandardScaler

In [58]:
scaler = StandardScaler()

Como siempre en Sklearn, tenemos que hacer un fit con nuestros datos de entrenamiento a el objeto.

Hacer un fit a el scaler con los datos NUMERICOS de train:

In [59]:
columnas_numericas = ["bill_length_mm",	"bill_depth_mm",	"flipper_length_mm",	"body_mass_g"]

In [60]:
scaler.fit(X_train[columnas_numericas])

StandardScaler(copy=True, with_mean=True, with_std=True)

Ahora, con el scaler podemos transformar los datos tanto en train como en test.

Transformar los datos numéricos de train (aplicar el scaler):

In [61]:
X_train[columnas_numericas] = scaler.transform(X_train[columnas_numericas])

In [26]:
X_train.head()

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
253,Biscoe,2.871447,-0.102020,2.077011,2.270111,Male
79,Torgersen,-0.350822,0.965417,-0.440463,-0.264906,Male
283,Biscoe,1.895560,-0.762814,2.148939,1.775474,Male
206,Dream,-0.277170,0.050471,-1.015885,-1.068692,Female
328,Biscoe,-0.129867,-1.626929,0.494599,0.446135,Female


Ahora nos quedan 2 variables categóricas, vamos a aplicar one hot encoder.

Recuerden que el fit se hace sobre los datos de entrenamiento y luego sobre los datos de test aplicamos únicamente transform.

Importar one hot encoder:

In [27]:
from sklearn.preprocessing import OneHotEncoder

Instanciar one hot encoder para cada variable categórica:

In [28]:
ohe_island = OneHotEncoder(sparse=False)
ohe_sex = OneHotEncoder(sparse=False)

Hacer fit con los datos de entrenamiento para ambos encoders:

In [29]:
ohe_island.fit(X_train[["island"]])
ohe_sex.fit(X_train[["sex"]])

OneHotEncoder(categories='auto', drop=None, dtype=<class 'numpy.float64'>,
              handle_unknown='error', sparse=False)

Obtener las variables con one hot encoded para ambas variables categóricas:

In [30]:
islands = ohe_island.transform(X_train[["island"]])
encoded_island_df = pd.DataFrame(data=islands, columns= ohe_island.get_feature_names())

sex = ohe_sex.transform(X_train[["sex"]])
encoded_sex_df = pd.DataFrame(data=sex, columns= ohe_sex.get_feature_names())

Hacer concat con X_train:

In [31]:
X_train.head()

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex
253,Biscoe,2.871447,-0.102020,2.077011,2.270111,Male
79,Torgersen,-0.350822,0.965417,-0.440463,-0.264906,Male
283,Biscoe,1.895560,-0.762814,2.148939,1.775474,Male
206,Dream,-0.277170,0.050471,-1.015885,-1.068692,Female
328,Biscoe,-0.129867,-1.626929,0.494599,0.446135,Female


In [32]:
X_train = pd.concat([X_train.reset_index(drop=True), encoded_island_df, encoded_sex_df], axis=1)
X_train.head()

,island,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,sex,x0_Biscoe,x0_Dream,x0_Torgersen,x0_Female,x0_Male
0,Biscoe,2.871447,-0.102020,2.077011,2.270111,Male,1.0,0.0,0.0,0.0,1.0
1,Torgersen,-0.350822,0.965417,-0.440463,-0.264906,Male,0.0,0.0,1.0,0.0,1.0
2,Biscoe,1.895560,-0.762814,2.148939,1.775474,Male,1.0,0.0,0.0,0.0,1.0
3,Dream,-0.277170,0.050471,-1.015885,-1.068692,Female,0.0,1.0,0.0,1.0,0.0
4,Biscoe,-0.129867,-1.626929,0.494599,0.446135,Female,1.0,0.0,0.0,1.0,0.0


descartar las columnas originales:

In [33]:
X_train = X_train.drop(["island", "sex"], axis=1)
X_train.head()

,bill_length_mm,bill_depth_mm,flipper_length_mm,body_mass_g,x0_Biscoe,x0_Dream,x0_Torgersen,x0_Female,x0_Male
0,2.871447,-0.102020,2.077011,2.270111,1.0,0.0,0.0,0.0,1.0
1,-0.350822,0.965417,-0.440463,-0.264906,0.0,0.0,1.0,0.0,1.0
2,1.895560,-0.762814,2.148939,1.775474,1.0,0.0,0.0,0.0,1.0
3,-0.277170,0.050471,-1.015885,-1.068692,0.0,1.0,0.0,1.0,0.0
4,-0.129867,-1.626929,0.494599,0.446135,1.0,0.0,0.0,1.0,0.0


## KNN

Ahora, con nuestro dataset limpio, entrenemos un KNN classifier.

Primero, importar knn classifier de sklearn:

In [34]:
from sklearn.neighbors import KNeighborsClassifier

Instanciar un KNN con n_neighbors = 5 y weights="uniform".

INVESTIGAR: ¿Qué significa weights = "uniform" ???

In [35]:
knn = KNeighborsClassifier(n_neighbors=5, weights='uniform')

Entrenar el modelo con los datos de entrenamiento:

In [36]:
knn.fit(X_train, y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='minkowski',
                     metric_params=None, n_jobs=None, n_neighbors=5, p=2,
                     weights='uniform')

Generar las predicciones para train y test. Tener en cuenta que para generar las de test, debemos aplicar el preprocesamiento a los datos (OHE y scaler)

In [37]:
# Preprocesamiento y predicciones de test:
X_test[columnas_numericas] = scaler.transform(X_test[columnas_numericas])
islands = ohe_island.transform(X_test[["island"]])
encoded_island_df = pd.DataFrame(data=islands, columns= ohe_island.get_feature_names())

sex = ohe_sex.transform(X_test[["sex"]])
encoded_sex_df = pd.DataFrame(data=sex, columns= ohe_sex.get_feature_names())

X_test = pd.concat([X_test.reset_index(drop=True), encoded_island_df, encoded_sex_df], axis=1)

X_test = X_test.drop(["island", "sex"], axis=1)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  
/usr/local/lib/python3.7/dist-packages/pandas/core/indexing.py:1734: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  isetter(loc, value[:, i].tolist())


In [38]:
pred_train = knn.predict(X_train)
pred_test = knn.predict(X_test)

Medir accuracy_score para train y test.

In [39]:
accuracy_score(y_train, pred_train)

0.9929328621908127

In [40]:
accuracy_score(y_test, pred_test)

1.0